### INFO 5502 Final Exam (05/12/2022, 240 points in total)

<span style='background:yellow'>**Question 3 (100 pt, each sub-question is 10 pt).**</span> 

Part A: Given Inetd dataset, an intrusion detection dataset that contains 2 columns. The first column represents a process ID number (PID) and the second column represents a system call. The goal is to extract the entire system call sequence by its PID, then tokenize and parse each sequence into 25-grams. After this, we’ll split the data into training and testing sets. Next, we ensure data quality in the training dataset by removing duplicate sequences within each class and overlapped sequences between both classes. To ensure a fair evaluation, we also clean the testing set by repeating the previous step. You will implement a Decision Tree model to train and test on the training set and testing set that we just clean. 


Part B: As the data type is sequential, we want to know how similar the sequences from both classes are and visualize how the similarity scores vary in different sequence lengths. Compute a cosine similarity score of 100 random pairs from both classes and get a median cosine similarity. Repeat this step in different sequence lengths (5, 10, 15, and 20). Plot a line chart to visualize how the median cosine similarities vary through sequence lengths of 5 to 25. 

You can download the datasets from github: https://github.com/unt-iialab/info5502-spring2022/tree/main/finalexam/q4


Step by step instructions and questions:

(1) Attached are the Normal and Intrusion data folders. Read in the data and store them in two separate dataframes. Combine the system calls (the second column) from the same PID (the first column) into a sequence or a string. 

(2) Parse an entire sequence into smaller sequences of size 25. (Hint: Use word_tokenize and ngrams from nltk library to parse).

(3) Add labels to the data, and partition the labeled data into training data and testing data with a ratio of 70/30.

(4) As the dataset has a lot of duplicate sequences, make sure that the testing data only contain unique sequences and that these sequences do not exist in the training data. Condense training data and testing data into two sets. Take the difference between the testing set and training set to remove the overlapped sequences from the testing data. 

(5) The sequences from each class can be repetitive, and some of these sequences may have different labels. Therefore, we need to remove the duplicate sequences within each class and the overlapped sequences from the Intrusion class. Start out by separating the Normal class and Intrusion class in the training data. Use the same concept in Question 4 to remove duplicate sequences and overlapped sequences from the training data.

(6) Repeat Question 5 to clean the testing data.

(7) Bootstrap the training data to get the same number of sequences from each class. Train and evaluate a Decision Tree model using the training data and testing data that we just clean. How is the performance?

(8) Randomly sample 100 pairs of sequences from both classes and calculate the median cosine similarity.

(9) Refer back to Question 2, parse the data into a length of 5, 10, 15, and 20. Clean the data from each class using Question 5. For each sequence length, calculate the median cosine similarity of 100 random pairs of sequences from both classes. 

(10) Plot a line chart to visualize how the median cosine similarities vary through different sequence lengths from 5 to 25.


In [1]:
#### You code here

import pandas as pd
import numpy as np

df_inetd=pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int",header=None,sep='\s+')
#df_inetd.head()

df_inetd.columns=["Pid","System_call"]
df_inetd.head()

#df_normal=pd.read_csv("https://github.com/unt-iialab/info5502-spring2022/blob/main/finalexam/q4/inetd-normal.int",sep=" ",header=None)


#df_normal.columns=["Pid","System_call"]
#df_normal.head()


,Pid,System_call
0,167,90
1,167,125
2,167,125
3,167,106
4,167,5


In [2]:
normal_df = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int", sep = " ", header = None)
normal_df.columns=["Pid","System_call"]
normal_df.head()

,Pid,System_call
0,167,90
1,167,125
2,167,125
3,167,106
4,167,5


In [4]:
intrusion_dict = {}
normal_dict = {}

# combining system calls
for pid in list(set(df_inetd['Pid'].values)):
    intrusion_dict[pid] = " ".join(str(value) for value in list(df_inetd[df_inetd['Pid'] == pid]['System_call'].values))

for pid in list(set(normal_df['Pid'].values)):
    normal_dict[pid] = " ".join(str(value) for value in list(normal_df[normal_df['Pid'] == pid]['System_call'].values))

intrusion_dict[167]


'90 125 125 106 5 90 6 5 3 90 90 90 90 6 125 91 125 125 125 136 49 24 47 50 45 45 5 106 106 106 106 106 2 1'

In [5]:
import re
from nltk.util import ngrams

i_list = []
n_list = []


for key in intrusion_dict.keys():
    s = intrusion_dict[key]
    tokens = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokens, 25)):
        i_list.append(ele)
        
for key in normal_dict.keys():
    s = normal_dict[key]
    tokens = [token for token in s.split(" ") if token != ""]
    
    for ele in list(ngrams(tokens, 25)):
        n_list.append(ele) 

print(len(i_list))
print(len(n_list))

7627
469


In [6]:
intrusion_df = pd.DataFrame(i_list)
normal_df = pd.DataFrame(n_list)

# adding labels to intrusion and normal data sequences
intrusion_df['class'] = 1
normal_df['class'] = 0

print(intrusion_df.shape)
print(normal_df.shape)

(7627, 26)
(469, 26)


In [7]:
combined_df = pd.DataFrame()

combined_df = combined_df.append(intrusion_df)
combined_df = combined_df.append(normal_df)

print(combined_df.shape)

(8096, 26)


C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\3216987951.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(intrusion_df)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\3216987951.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(normal_df)


In [8]:
combined_df = combined_df.drop_duplicates(subset = combined_df.columns[:-1], keep = 'last')
combined_df.shape

C:\Users\lalit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:6250: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)


(333, 26)

In [9]:
combined_df['class'].value_counts()
combined_df.columns[:-1]

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24],
      dtype='object')

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X = combined_df.drop("class", axis = 1)
y = combined_df['class']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2, random_state = 0)

dtclassifier = DecisionTreeClassifier(max_depth = 10)
dtclassifier.fit(X_train, y_train)
y_pred = dtclassifier.predict(X_valid)

print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.82      0.78        22
           1       0.91      0.87      0.89        45

    accuracy                           0.85        67
   macro avg       0.83      0.84      0.83        67
weighted avg       0.86      0.85      0.85        67



In [11]:
import re
from nltk.util import ngrams
import pandas as pd

cosine_similar_ngram = {}

for ngram in range(5, 26, 5):

    intrusion_data = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-intrusion.int", sep = " ", header = None)
    normal_data = pd.read_csv("https://raw.githubusercontent.com/unt-iialab/info5502-spring2022/main/finalexam/q4/inetd-normal.int", sep = " ", header = None)

    intrusion_data.columns = ['PID', 'SystemCalls']
    normal_data.columns = ['PID', 'SystemCalls']

    intrusion_dict = {}
    normal_dict = {}

    for pid in list(set(intrusion_data['PID'].values)):
        intrusion_dict[pid] = " ".join(str(value) for value in list(intrusion_data[intrusion_data['PID'] == pid]['SystemCalls'].values))

    for pid in list(set(normal_data['PID'].values)):
        normal_dict[pid] = " ".join(str(value) for value in list(normal_data[normal_data['PID'] == pid]['SystemCalls'].values))

    intrusion_list = []
    normal_list = []
    for key in intrusion_dict.keys():
        s = intrusion_dict[key]
        tokens = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(tokens, ngram)):
            intrusion_list.append(ele)

    for key in normal_dict.keys():
        s = normal_dict[key]
        tokens = [token for token in s.split(" ") if token != ""]

        for ele in list(ngrams(tokens, ngram)):
            normal_list.append(ele)

    intrusion_df = pd.DataFrame(intrusion_list)
    normal_df = pd.DataFrame(normal_list)

    intrusion_df['class'] = 1
    normal_df['class'] = 0

    combined_df = pd.DataFrame()

    combined_df = combined_df.append(intrusion_df)
    combined_df = combined_df.append(normal_df)

    print(combined_df.shape)
    print()


    combined_df = combined_df.drop_duplicates(subset = combined_df.columns[:-1], keep = 'last')
    print(combined_df.shape)
    print()

    from sklearn.metrics.pairwise import cosine_similarity
    import statistics

    similarities_list = []
    for i in range(0, 100):
        list1 = list(combined_df[combined_df['class'] == 0].sample(n = 1).iloc[0, :].values)
        list2 = list(combined_df[combined_df['class'] == 1].sample(n = 1).iloc[0, :].values)
        try:
            similarities_list.append(cosine_similarity([list1], [list2]))
        except Exception as e:
            print(e)
            print(list1, list2)

    print(statistics.median(similarities_list)[0][0])
    print()
    cosine_similar_ngram[ngram] = statistics.median(similarities_list)[0][0]

C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(intrusion_df)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(normal_df)
C:\Users\lalit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:6250: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combi

(8776, 6)

(168, 6)

0.7820327756782337

(8606, 11)

(213, 11)

0.7151156189452039

(8436, 16)

(254, 16)

0.7249165541061693



C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(intrusion_df)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(normal_df)
C:\Users\lalit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:6250: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combi

(8266, 21)

(293, 21)

0.7225481830982801

(8096, 26)

(333, 26)

0.7231744985521069



C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(intrusion_df)
C:\Users\lalit\AppData\Local\Temp\ipykernel_4932\1993428251.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(normal_df)
C:\Users\lalit\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:6250: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  diff = Index(subset).difference(self.columns)
